# Text2Everything SDK - Feature Guide

**Version:** 0.1.7  
**Updated:** October 24, 2025

This comprehensive guide covers all the advanced features introduced in the Text2Everything Python SDK v0.1.7, which achieves 100% API parity.

## Table of Contents

1. [Setup & Configuration](#setup)
2. [Bulk Operations](#bulk-operations)
3. [Chat Presets](#chat-presets)
4. [Advanced Features](#advanced-features)
5. [Best Practices & Tips](#best-practices)

---

## 1. Setup & Configuration <a id="setup"></a>

First, let's set up the SDK client with proper configuration.

In [ ]:
# Import the SDK
from text2everything_sdk import Text2EverythingClient
import os

# Initialize the client
# The SDK automatically loads from .env file if present
client = Text2EverythingClient(
    base_url=os.getenv("T2E_BASE_URL", "http://localhost:80"),
    access_token=os.getenv("T2E_ACCESS_TOKEN"),
    workspace_name=os.getenv("T2E_WORKSPACE_NAME")
)

print("✅ Client initialized successfully")
print(f"Base URL: {client.base_url}")
print(f"Workspace: {client.workspace_name}")

In [ ]:
# Create or use existing project for this guide
PROJECT_NAME = "SDK Feature Guide Demo"

# Try to find existing project
projects = client.projects.list()
demo_project = next((p for p in projects if p.name == PROJECT_NAME), None)

if demo_project:
    project_id = demo_project.id
    print(f"✅ Using existing project: {project_id}")
else:
    # Create new project
    demo_project = client.projects.create(
        name=PROJECT_NAME,
        description="Demo project for SDK feature guide"
    )
    project_id = demo_project.id
    print(f"✅ Created new project: {project_id}")

print(f"\nProject Name: {demo_project.name}")
print(f"Project ID: {project_id}")

---

## 2. Bulk Operations <a id="bulk-operations"></a>

Bulk delete operations allow you to efficiently manage multiple resources at once. This is particularly useful for:
- Cleaning up test data
- Managing large datasets
- Batch resource management

All bulk delete operations return a structured response with:
- `deleted_count`: Number of successfully deleted items
- `failed_ids`: List of IDs that failed to delete (if any)

### 2.1 Bulk Delete Contexts

In [ ]:
# Create some test contexts
print("Creating test contexts...")
context_ids = []

for i in range(5):
    context = client.contexts.create(
        project_id=project_id,
        name=f"Bulk Delete Test Context {i+1}",
        content=f"This is test context #{i+1} for demonstrating bulk delete",
        is_always_displayed=False
    )
    context_ids.append(context.id)
    print(f"  ✓ Created context {i+1}: {context.id}")

print(f"\n✅ Created {len(context_ids)} test contexts")

In [ ]:
# Bulk delete the contexts
print("Performing bulk delete...")
result = client.contexts.bulk_delete(
    project_id=project_id,
    context_ids=context_ids
)

print(f"\n✅ Bulk Delete Results:")
print(f"   Deleted: {result['deleted_count']} contexts")
print(f"   Failed: {len(result.get('failed_ids', []))} contexts")

if result.get('failed_ids'):
    print(f"   Failed IDs: {result['failed_ids']}")

### 2.2 Bulk Delete Schema Metadata

Schema metadata bulk delete automatically handles split groups - when you delete any part of a split group, all parts are deleted.

In [ ]:
# Create test schemas
print("Creating test schemas...")
schema_ids = []

for i in range(3):
    schema = client.schema_metadata.create(
        project_id=project_id,
        name=f"Test Table {i+1}",
        description=f"Test schema for bulk delete demo",
        schema_data={
            "table": {
                "name": f"test_table_{i+1}",
                "columns": [
                    {"name": "id", "type": "INTEGER"},
                    {"name": "value", "type": "VARCHAR(100)"}
                ]
            }
        }
    )
    schema_ids.append(schema.id)
    print(f"  ✓ Created schema {i+1}: {schema.id}")

# Bulk delete
result = client.schema_metadata.bulk_delete(
    project_id=project_id,
    schema_ids=schema_ids
)

print(f"\n✅ Deleted {result['deleted_count']} schemas")

### 2.3 Bulk Delete Golden Examples

In [ ]:
# Create test golden examples
print("Creating test golden examples...")
example_ids = []

test_examples = [
    {"query": "Show me all users", "sql": "SELECT * FROM users"},
    {"query": "Count active users", "sql": "SELECT COUNT(*) FROM users WHERE active = true"},
    {"query": "Top 10 by revenue", "sql": "SELECT * FROM orders ORDER BY revenue DESC LIMIT 10"}
]

for i, ex in enumerate(test_examples):
    example = client.golden_examples.create(
        project_id=project_id,
        name=f"Example {i+1}",
        user_query=ex["query"],
        sql_query=ex["sql"],
        description="Test example for bulk delete"
    )
    example_ids.append(example.id)
    print(f"  ✓ Created example {i+1}")

# Bulk delete
result = client.golden_examples.bulk_delete(
    project_id=project_id,
    example_ids=example_ids
)

print(f"\n✅ Deleted {result['deleted_count']} golden examples")

### 2.4 Bulk Delete Feedback

In [ ]:
# Note: Feedback requires a chat message ID
# This is a demonstration of the API - actual implementation would need
# a real chat session and message

print("ℹ️  Feedback bulk delete requires chat messages")
print("   Skipping this example - see README for implementation details")

# Example code (commented):
# feedback_ids = ["fb_id_1", "fb_id_2", "fb_id_3"]
# result = client.feedback.bulk_delete(
#     project_id=project_id,
#     feedback_ids=feedback_ids
# )

---

## 3. Chat Presets <a id="chat-presets"></a>

Chat presets allow you to create reusable chat configurations with:
- Predefined connectors
- System prompts
- Cutoff thresholds for contexts, schemas, feedback, and examples
- Prompt templates

Benefits:
- **Consistency**: Same settings across multiple chat sessions
- **Efficiency**: Quick setup for common use cases
- **Flexibility**: Easy switching between different configurations

### 3.1 Creating Chat Presets

In [ ]:
# First, we need a connector
# Check if we have any connectors
try:
    connectors = client.connectors.list(project_id=project_id)
    if connectors:
        connector_id = connectors[0].id
        print(f"✅ Using existing connector: {connector_id}")
    else:
        print("⚠️  No connectors found - you'll need to create one first")
        print("   Skipping preset creation for now")
        connector_id = None
except Exception as e:
    print(f"ℹ️  Could not list connectors: {e}")
    connector_id = None

In [ ]:
# Create a chat preset (if we have a connector)
if connector_id:
    # First, create a prompt template (recommended approach)
    template = client.chat_presets.create_prompt_template(
        project_id=project_id,
        name="Production Analytics Template",
        system_prompt="You are an expert data analyst. Provide accurate SQL queries based on the schema.",
        description="Template for production analytics"
    )
    template_id = template["id"]
    print(f"✅ Created prompt template: {template_id}")
    
    # Create preset with the template
    response = client.chat_presets.create(
        project_id=project_id,
        name="Production Analytics",
        collection_name="production_analytics_collection",
        description="Preset for production data analysis",
        connector_id=connector_id,
        prompt_template_id=template_id,
        api_system_prompt="Provide detailed explanations with your SQL queries.",
        chat_settings={
            "llm": "gpt-4",
            "include_chat_history": "auto"
        }
    )
    
    collection_id = response.collection_id
    print(f"\n✅ Created preset: {response.collection_name}")
    print(f"   Collection ID: {collection_id}")
    print(f"   Template ID: {response.prompt_template_id}")
    
    # Get full preset details
    preset = client.chat_presets.get(project_id, collection_id)
    preset_id = preset.id
    print(f"   Preset ID: {preset_id}")
else:
    print("ℹ️  Skipping preset creation - no connector available")
    preset_id = None
    collection_id = None

### 3.2 Managing Chat Presets

In [ ]:
# List all presets
presets = client.chat_presets.list(project_id=project_id)
print(f"Found {len(presets)} chat presets:")
for p in presets:
    print(f"  • {p.name} ({p.id})")
    print(f"    Description: {p.description}")
    print(f"    Active: {p.is_active}")

In [ ]:
# Update a preset (if we have one)
if collection_id:
    updated = client.chat_presets.update(
        project_id=project_id,
        collection_id=collection_id,
        description="Updated: Production analytics with enhanced settings",
        chat_settings={
            "llm": "gpt-4-turbo",
            "include_chat_history": "true"
        }
    )
    print(f"✅ Updated preset")
    print(f"   Collection ID: {updated.collection_id}")
    
    # Get updated preset to see changes
    preset = client.chat_presets.get(project_id, collection_id)
    print(f"   New description: {preset.description}")

### 3.3 Prompt Templates

In [ ]:
# Add prompt template to preset
if preset_id:
    template = client.chat_presets.add_prompt_template(
        project_id=project_id,
        preset_id=preset_id,
        template_name="Detailed Analysis",
        template_content="Analyze the following query and provide: 1) SQL query, 2) Explanation, 3) Performance considerations\n\nQuery: {query}"
    )
    print(f"✅ Added prompt template: {template.template_name}")
    
    # List templates
    templates = client.chat_presets.list_prompt_templates(
        project_id=project_id,
        preset_id=preset_id
    )
    print(f"\nTotal templates: {len(templates)}")

### 3.4 Activating and Using Presets

In [ ]:
# Activate a preset (uses preset_id, not collection_id)
if preset_id:
    activated = client.chat_presets.activate(
        project_id=project_id,
        preset_id=preset_id
    )
    print(f"✅ Activated preset: {activated.name}")
    print(f"   Active: {activated.is_active}")
    
    # Get active preset
    active_preset = client.chat_presets.get_active(project_id=project_id)
    if active_preset:
        print(f"\nActive preset: {active_preset.name}")
        print(f"   Connector: {active_preset.connector_id}")
        print(f"   Collection ID: {active_preset.h2ogpte_collection_id}")
    else:
        print("\nNo active preset")

In [ ]:
# Create chat session from preset
if preset_id:
    session = client.chat_sessions.create_from_preset(
        project_id=project_id,
        preset_id=preset_id
    )
    print(f"✅ Created chat session from preset: {session.id}")
    print(f"   Inherits all preset settings")
    
    # Or use active preset
    # session = client.chat_sessions.create_from_active_preset(project_id=project_id)

### 3.5 Inline Template Creation (Current Limitation)

**Note:** The `prompt_template` parameter (for inline template creation) is accepted for API parity but not currently processed by the API.

**Recommended Approach:** Create templates separately using `create_prompt_template()`, then reference by ID (as shown in Section 3.1).

In [ ]:
# This demonstrates the current API limitation
if connector_id:
    print("ℹ️  Demonstrating inline template limitation...")
    
    # This will be accepted but template won't be created
    response = client.chat_presets.create(
        project_id=project_id,
        name="Inline Template Test",
        collection_name="inline_test_collection",
        connector_id=connector_id,
        prompt_template={
            "name": "Inline Template",
            "system_prompt": "This is an inline template",
            "description": "Test inline creation"
        }
    )
    
    # Check if template was created
    preset = client.chat_presets.get(project_id, response.collection_id)
    if preset.prompt_template_id:
        print(f"   ✓ Template ID populated: {preset.prompt_template_id}")
    else:
        print(f"   ⚠️  Template not created (expected - known limitation)")
        print(f"   Use create_prompt_template() instead")

---

## 4. Advanced Features <a id="advanced-features"></a>

These features provide deeper integration with the Text2Everything system and enable performance optimizations.

### 4.1 Project Collections

Collections represent the H2OGPTE storage layer for different resource types. Each resource type (contexts, schemas, etc.) has its own collection.

In [ ]:
# List all collections
collections = client.projects.list_collections(project_id=project_id)

print(f"Found {len(collections)} collections:")
for collection in collections:
    print(f"\n  • Type: {collection.component_type}")
    print(f"    Collection ID: {collection.h2ogpte_collection_id}")
    print(f"    Created: {collection.created_at}")

In [ ]:
# Get specific collection by type
try:
    contexts_collection = client.projects.get_collection_by_type(
        project_id=project_id,
        component_type="contexts"
    )
    print(f"✅ Contexts Collection:")
    print(f"   ID: {contexts_collection.h2ogpte_collection_id}")
    print(f"   Type: {contexts_collection.component_type}")
except Exception as e:
    print(f"ℹ️  No contexts collection yet: {e}")

### 4.2 Execution Cache Lookup

Query the execution cache to find similar past queries. This enables:
- Performance optimization (reuse cached results)
- Learning from past queries
- Quality improvement (find positively-rated executions)

In [ ]:
# Look up cached executions
if connector_id:
    cache_result = client.chat.execution_cache_lookup(
        project_id=project_id,
        user_query="Show me the top 10 customers by revenue",
        connector_id=connector_id,
        similarity_threshold=0.7,  # 70% similarity required
        top_n=5,  # Return top 5 matches
        only_positive_feedback=True  # Only positively rated
    )
    
    print(f"Cache Lookup Results:")
    print(f"  Cache Hit: {cache_result.cache_hit}")
    print(f"  Candidates Checked: {cache_result.candidates_checked}")
    print(f"  Matches Found: {len(cache_result.matches)}")
    
    if cache_result.cache_hit:
        print(f"\n  Top Matches:")
        for i, match in enumerate(cache_result.matches[:3], 1):
            print(f"\n  {i}. Similarity: {match.similarity_score:.2f}")
            print(f"     Query: {match.execution.user_query}")
            print(f"     SQL: {match.execution.sql_query[:100]}...")
else:
    print("ℹ️  Skipping cache lookup - no connector available")

### 4.3 Schema Split Groups

When you create a schema with more than 8 columns, it's automatically split into multiple parts for better performance. The SDK handles this transparently.

In [ ]:
# Create a large schema (>8 columns)
large_schema = client.schema_metadata.create(
    project_id=project_id,
    name="Large Customer Table",
    description="Customer table with many columns",
    schema_data={
        "table": {
            "name": "customers_large",
            "columns": [
                {"name": "id", "type": "INTEGER", "description": "Customer ID"},
                {"name": "first_name", "type": "VARCHAR(50)", "description": "First name"},
                {"name": "last_name", "type": "VARCHAR(50)", "description": "Last name"},
                {"name": "email", "type": "VARCHAR(255)", "description": "Email address"},
                {"name": "phone", "type": "VARCHAR(20)", "description": "Phone number"},
                {"name": "address", "type": "VARCHAR(255)", "description": "Street address"},
                {"name": "city", "type": "VARCHAR(100)", "description": "City"},
                {"name": "state", "type": "VARCHAR(2)", "description": "State code"},
                {"name": "zip", "type": "VARCHAR(10)", "description": "ZIP code"},
                {"name": "country", "type": "VARCHAR(100)", "description": "Country"},
                {"name": "created_at", "type": "TIMESTAMP", "description": "Creation date"},
                {"name": "updated_at", "type": "TIMESTAMP", "description": "Last update"},
            ]
        }
    }
)

print(f"✅ Created schema: {large_schema.name}")
print(f"   ID: {large_schema.id}")

# Check if it was split
if large_schema.split_group_id:
    print(f"\n📊 Schema was split:")
    print(f"   Split Group ID: {large_schema.split_group_id}")
    print(f"   This part: {large_schema.split_index}/{large_schema.total_splits}")
    
    # Retrieve the complete split group
    split_group = client.schema_metadata.get_split_group(
        project_id=project_id,
        split_group_id=large_schema.split_group_id
    )
    
    print(f"\n   All parts in group: {split_group['total_parts']}")
    for part in split_group['parts']:
        print(f"     • Part {part.split_index}: {part.id}")
else:
    print(f"\nℹ️  Schema was not split (≤8 columns or split disabled)")

---

## 5. Best Practices & Tips <a id="best-practices"></a>

### 5.1 Bulk Operations

**When to use:**
- Cleaning up test data after development
- Managing resources in batches
- Periodic cleanup operations

**Best practices:**
- Always check the result for failed_ids
- For large batches, consider chunking (50-100 items per batch)
- Handle failures appropriately in production code

### 5.2 Chat Presets

**When to use:**
- Different analysis scenarios (production, development, ad-hoc)
- Team-specific configurations
- Different data sources

**Best practices:**
- Use descriptive names for presets
- Document cutoff threshold choices
- Test presets before deploying to production
- Use prompt templates for consistency

### 5.3 Performance Tips

**Execution Cache:**
- Use cache lookup before generating new SQL
- Set appropriate similarity thresholds (0.7-0.9)
- Filter by positive feedback for quality

**Schema Management:**
- Large schemas are automatically split - no action needed
- Bulk delete handles split groups automatically
- Use collections API to monitor resource organization

### 5.4 Error Handling

Always implement proper error handling in production code

---

## Summary

This guide covered the major features introduced in SDK v0.1.7:

✅ **Bulk Operations** - Efficient multi-resource management  
✅ **Chat Presets** - Reusable chat configurations  
✅ **Project Collections** - H2OGPTE storage layer access  
✅ **Execution Cache** - Performance optimization through caching  
✅ **Schema Split Groups** - Automatic handling of large schemas  

For more information:
- [SDK README](README.md)
- [Quick Start Guide](T2E Quick start.ipynb)

---

**Version:** 0.1.7  
**Last Updated:** October 24, 2025